### Recurrent Neural Networks `RNN`
Recurrent Neural Networks (RNN) are designed to work with sequential data. Sequential data(can be time-series) can be in form of text, audio, video etc.
RNN uses the previous information in the sequence to produce the current output. Let's consider the following illustration suppose we have the sentence:
```
"I love deep learning."
```
* At `t0` the input will be `I`
* At `t1` the input will be `I love`
* At `t2` the input will be `I love deep`
* At `t3` the input will be `I love deep learning.`
> At the end the `NN` will have all the infomation about the last step.

<p align="center"><img src="https://miro.medium.com/max/480/1*gEA0-LTj05xtESA5XoBxPw.gif"/> </p>

**Note**: In ``RNN`` weights and bias for all the nodes in the layer are same. The default activation function of `RNN` is `tanh`.

> Let's have a look at a `RNN` unit.
<p align="center"><img src="https://miro.medium.com/max/332/0*eRJCRsikdGGu8ffA.png"/></p>

> ``RNN’s`` face **short-term memory** problem. It is caused due to ``vanishing gradient`` problem. As ``RNN`` processes more steps it suffers from vanishing gradient more than other neural network architectures.

### The vaishing gradient problem.
In ``RNN`` to train the network you backpropagate through time, at each step the gradient is calculated. The gradient is used to update weights in the network. If the effect of the previous layer on the current layer is small then the gradient value will be small and vice-versa. If the gradient of the previous layer is smaller, then the gradient of the current layer will be even smaller. This makes the gradients exponentially shrink down as we backpropagate. Smaller gradient means it will not affect the weight updation. Due to this, the network does not learn the effect of earlier inputs. Thus, causing the ``short-term`` memory problem.

To solve the problem of Vanishing gradient we use two specialised versions of RNN.

#### Gated Recurrent Units `GRU`
In this `RNN` there are two gates namely:
1. update gate
2. reset gate

<p align="center"><img src="https://miro.medium.com/max/700/1*RiOzdOVaaeKrUotY7-1a2A.png"/></p>

Gates are nothing but neural networks, each gate has its own weights and biases(but don’t forget that weights and bias for all nodes in one layer are same).

* **Update gate** -
Update gate decides if the cell state should be updated with the candidate state(current activation value)or not.
* **Reset gate** -
The reset gate is used to decide whether the previous cell state is important or not. Sometimes the reset gate is not used in simple GRU.
* **Candidate cell** -
It is just simply the same as the hidden state(activation) of RNN.
**Final cell state** -
The final cell state is dependent on the update gate. It may or may not be updated with candidate state. Remove some content from last cell state, and write some new cell content.

In GRU,


* If reset close to 0, ignore previous hidden state (allows the model to drop information that is irrelevant in the future).
* If gamma(update gate) close to 1, then we can copy information in that unit through many steps!
* Gamma Controls how much of past state should matter now.

#### Long Short-Term Memory `LSTM`
LSTMs are pretty much similar to GRU’s, they are also intended to solve the vanishing gradient problem. It has the following gates

1. update gate
2. reset gate
3. forget gate
4. output gate

<p align="center"><img src="https://miro.medium.com/max/700/1*lSDKRennQMpJFL4xxJHloQ.png"/></p>

> All 3 gates(input gate, output gate, forget gate) use sigmoid as activation function so all gate values are between 0 and 1.

* **Forget gate** -
It controls what is kept vs forgotten, from previous cell state. In laymen terms, it will decide how much information from the previous state should be kept and forget remaining.
* **Output gate** -
It controls which parts of the cell are output to the hidden state. It will determine what the next hidden state will be.

### Ref

[Article](https://medium.com/analytics-vidhya/rnn-vs-gru-vs-lstm-863b0b7b1573)


### Practical Example
We are going to use `RNN` with the `airline-passanger` dataset which we will load from a `csv` file. The dataset provides a record of the number of people travelling in US airlines in a particular month. we will only use the `Passengers` column.

### Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

### Data Prep

In [ ]:
data = pd.read_csv("airline-passengers.csv")
data.head()

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [ ]:
data.isnull().values.any()

False

In [ ]:
scaler = MinMaxScaler()
dataset = scaler.fit_transform(data["Passengers"].values.reshape(-1, 1))
train_size = int(len(dataset) * 0.75)
test_size = len(dataset) - train_size
train=dataset[:train_size,:]
test=dataset[train_size:142,:]
def getdata(data,lookback):
    X,Y=[],[]
    for i in range(len(data)-lookback-1):
        X.append(data[i:i+lookback,0])
        Y.append(data[i+lookback,0])
    return np.array(X),np.array(Y).reshape(-1,1)
lookback=1

X_train, y_train=getdata(train, lookback)
X_test,y_test=getdata(test,lookback)
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

### ``SimpleRNN``

In [ ]:
X_train.shape

(106, 1, 1)

In [ ]:
model_1 = keras.Sequential([
    keras.layers.SimpleRNN(64, input_shape=(1, 1)),
    keras.layers.Dense(1, activation='softmax')
], name="model_1")

model_1.compile(
    loss='mean_squared_error',optimizer='adam',
    metrics=['mse']
)
model_1.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)

Epoch 1/10
4/4 [==============================] - 2s 153ms/step - loss: 0.5928 - mse: 0.5928 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 2/10
4/4 [==============================] - 0s 23ms/step - loss: 0.6040 - mse: 0.6040 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 3/10
4/4 [==============================] - 0s 20ms/step - loss: 0.5992 - mse: 0.5992 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 4/10
4/4 [==============================] - 0s 20ms/step - loss: 0.5815 - mse: 0.5815 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 5/10
4/4 [==============================] - 0s 21ms/step - loss: 0.6060 - mse: 0.6060 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 6/10
4/4 [==============================] - 0s 23ms/step - loss: 0.6001 - mse: 0.6001 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 7/10
4/4 [==============================] - 0s 23ms/step - loss: 0.5900 - mse: 0.5900 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 8/10
4/4 [==============================] - 0s 22ms/step - loss: 0.6034 - mse: 0.6034 - va

> `LSTM`

In [ ]:
model_2 = keras.Sequential([
    keras.layers.LSTM(64, input_shape=(1, 1)),
    keras.layers.Dense(1, activation='softmax')
], name="model_2")

model_2.compile(
    loss='mean_squared_error',optimizer='adam',
    metrics=['mse']
)
model_2.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)

Epoch 1/10
4/4 [==============================] - 4s 318ms/step - loss: 0.5990 - mse: 0.5990 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 2/10
4/4 [==============================] - 0s 25ms/step - loss: 0.6028 - mse: 0.6028 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 3/10
4/4 [==============================] - 0s 23ms/step - loss: 0.5872 - mse: 0.5872 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 4/10
4/4 [==============================] - 0s 23ms/step - loss: 0.6126 - mse: 0.6126 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 5/10
4/4 [==============================] - 0s 22ms/step - loss: 0.5921 - mse: 0.5921 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 6/10
4/4 [==============================] - 0s 23ms/step - loss: 0.5836 - mse: 0.5836 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 7/10
4/4 [==============================] - 0s 23ms/step - loss: 0.5894 - mse: 0.5894 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 8/10
4/4 [==============================] - 0s 23ms/step - loss: 0.5768 - mse: 0.5768 - va

In [ ]:
model_3 = keras.Sequential([
    keras.layers.GRU(64, input_shape=(1, 1)),
    keras.layers.Dense(1, activation='softmax')
], name="model_3")

model_3.compile(
    loss='mean_squared_error',optimizer='adam',
    metrics=['mse']
)
model_3.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)

Epoch 1/10
4/4 [==============================] - 6s 290ms/step - loss: 0.6069 - mse: 0.6069 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 2/10
4/4 [==============================] - 0s 23ms/step - loss: 0.6142 - mse: 0.6142 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 3/10
4/4 [==============================] - 0s 23ms/step - loss: 0.5986 - mse: 0.5986 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 4/10
4/4 [==============================] - 0s 24ms/step - loss: 0.5898 - mse: 0.5898 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 5/10
4/4 [==============================] - 0s 22ms/step - loss: 0.6033 - mse: 0.6033 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 6/10
4/4 [==============================] - 0s 23ms/step - loss: 0.6130 - mse: 0.6130 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 7/10
4/4 [==============================] - 0s 23ms/step - loss: 0.5835 - mse: 0.5835 - val_loss: 0.1599 - val_mse: 0.1599
Epoch 8/10
4/4 [==============================] - 0s 22ms/step - loss: 0.5692 - mse: 0.5692 - va

> This Notebook is more focusing on the theory and how to use `RNN` not about training or testing `metrics`

### Nesting `RNN's`
To nest `RNN` we should just pass `return_sequences =True` to `RNN` layers for example:

> GRU
```python
keras.layers.Sequential([
    keras.layers.GRU(64, input_shape=(1, 1), return_sequences=True),
    keras.layers.GRU(64, return_sequences=True),
    ...
])
```

> RNN / SimpleRNN
```python
keras.layers.Sequential([
    keras.layers.SimpleRNN(64, input_shape=(1, 1), return_sequences=True),
    keras.layers.SimpleRNN(64, return_sequences=True),
    ...
])
```

> LSTM

```python
keras.layers.Sequential([
    keras.layers.LSTM(64, input_shape=(1, 1), return_sequences=True),
    keras.layers.LSTM(64, return_sequences=True),
    ...
])
```
> We can also have different `RNN` layers nested for example.

```python
keras.layers.Sequential([
    keras.layers.GRU(64, input_shape=(1, 1), return_sequences=True),
    keras.layers.LTSM(64, return_sequences=True),
    ...
])
```

### Bidirectional `RNN`
Bidirectional layer is a layer that is used to wrap such as ``keras.layers.LSTM`` or ``keras.layers.GRU.`` It can also be a `keras.layes.Layer` tha meet [these](https://keras.io/api/layers/recurrent_layers/bidirectional/) conditions.

To enable straight (past) and reverse traversal of input (future), Bidirectional RNNs, or BRNNs, are used. A BRNN is a combination of two RNNs - one RNN moves forward, beginning from the start of the data sequence, and the other, moves backward, beginning from the end of the data sequence. The network blocks in a BRNN can either be simple RNNs, GRUs, or LSTMs.

<p>
<img src="https://lh5.googleusercontent.com/glWMVLJyG4ZSLidw9Bb0d58IvJccRdD-Hb8Luw1nHB8tsamrGXl2oeC1dk7dhIhnBnaPxbBnr2SysIjbgxOn9VbUw6EeCGNX66cYZF0EtslhRlucHYL5jSPcKLTAbnPQ79MT8A"/>
</p>

Applications:

* Handwriting Recognition
* Speech Recognition
* Dependency Parsing
* Natural Language Processing

* [Ref](https://blog.paperspace.com/bidirectional-rnn-keras/)

##### Usage

```python
tf.keras.layers.Bidirectional(
    layer, merge_mode="concat", weights=None, backward_layer=None, **kwargs
)
```

* [Docs](https://keras.io/api/layers/recurrent_layers/bidirectional/)

> Example.

In [ ]:
model = keras.Sequential([
    keras.layers.Bidirectional(
        keras.layers.LSTM(10, return_sequences=True), input_shape=(5, 10)
    ),
    keras.layers.Bidirectional(keras.layers.LSTM(10)),
    keras.layers.Dense(5)
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 5, 20)             1680      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 20)                2480      
_________________________________________________________________
dense (Dense)                (None, 5)                 105       
Total params: 4,265
Trainable params: 4,265
Non-trainable params: 0
_________________________________________________________________


> Example With custom ``backward`` layer

In [ ]:

forward_layer = keras.layers.LSTM(10, return_sequences=True)
backward_layer = keras.layers.LSTM(10, activation='relu', return_sequences=True,
                       go_backwards=True)

model = keras.Sequential([
     keras.layers.Bidirectional(forward_layer, backward_layer=backward_layer,
                         input_shape=(5, 10)),
      keras.layers.Dense(5)
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_3 (Bidirection (None, 5, 20)             1680      
_________________________________________________________________
dense_2 (Dense)              (None, 5, 5)              105       
Total params: 1,785
Trainable params: 1,785
Non-trainable params: 0
_________________________________________________________________


 > **Practical Example**: ``imdb`` dataset.

In [2]:
max_features = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review
(X_train, y_train), (X_val, y_val) = keras.datasets.imdb.load_data(
    num_words=max_features
)
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_val = keras.preprocessing.sequence.pad_sequences(X_val, maxlen=maxlen)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

> Building the model without `Bidirectional` layer.

In [4]:
input_layer = keras.layers.Input(shape=(None, ), dtype="int32")
x = keras.layers.Embedding(max_features, 128)(input_layer)
x = keras.layers.LSTM(256, return_sequences=True)(x)
x = keras.layers.LSTM(64)(x)
output_layer = keras.layers.Dense(1, activation='sigmoid')(x)
model_4 = keras.Model(inputs=input_layer, outputs=output_layer, name="model_4")
model_4.compile(
"adam", "binary_crossentropy", metrics=["accuracy"]
)
model_4.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val),verbose=1, batch_size=128 )

Epoch 1/5
196/196 [==============================] - 19s 86ms/step - loss: 0.4673 - accuracy: 0.7718 - val_loss: 0.3545 - val_accuracy: 0.8513
Epoch 2/5
196/196 [==============================] - 16s 83ms/step - loss: 0.2422 - accuracy: 0.9064 - val_loss: 0.3546 - val_accuracy: 0.8512
Epoch 3/5
196/196 [==============================] - 16s 82ms/step - loss: 0.1562 - accuracy: 0.9446 - val_loss: 0.3725 - val_accuracy: 0.8559
Epoch 4/5
196/196 [==============================] - 16s 82ms/step - loss: 0.1052 - accuracy: 0.9641 - val_loss: 0.4509 - val_accuracy: 0.8458
Epoch 5/5
196/196 [==============================] - 16s 82ms/step - loss: 0.0649 - accuracy: 0.9795 - val_loss: 0.5046 - val_accuracy: 0.8505


> Building the model with `Bidirectional` layer.

In [5]:
input_layer = keras.layers.Input(shape=(None, ), dtype="int32")
x = keras.layers.Embedding(max_features, 128)(input_layer)
x = keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True))(x)
x = keras.layers.Bidirectional(keras.layers.LSTM(64))(x)
output_layer = keras.layers.Dense(1, activation='sigmoid')(x)
model_4 = keras.Model(inputs=input_layer, outputs=output_layer, name="model_4")
model_4.compile(
"adam", "binary_crossentropy", metrics=["accuracy"]
)
model_4.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val),verbose=1, batch_size=64 )

Epoch 1/5
391/391 [==============================] - 46s 106ms/step - loss: 0.4783 - accuracy: 0.7732 - val_loss: 0.4038 - val_accuracy: 0.8142
Epoch 2/5
391/391 [==============================] - 40s 104ms/step - loss: 0.2961 - accuracy: 0.8800 - val_loss: 0.4209 - val_accuracy: 0.8157
Epoch 3/5
391/391 [==============================] - 41s 104ms/step - loss: 0.2188 - accuracy: 0.9171 - val_loss: 0.3803 - val_accuracy: 0.8413
Epoch 4/5
391/391 [==============================] - 40s 103ms/step - loss: 0.1514 - accuracy: 0.9448 - val_loss: 0.4186 - val_accuracy: 0.8544
Epoch 5/5
391/391 [==============================] - 40s 103ms/step - loss: 0.0961 - accuracy: 0.9680 - val_loss: 0.4512 - val_accuracy: 0.8493


> **Conclusion:** 
A Bidirectional RNN is a combination of two RNNs training the network in opposite directions, one from the beginning to the end of a sequence, and the other, from the end to the beginning of a sequence. It helps in analyzing the future events by not limiting the model's learning to past and present.